In [2]:
#TODO bonito only basecalls DNA not RNA?
#TODO load the actual models - install developer version
#TODO install setup.py in develop version of bonito
import os
os.environ.get('CUDA_VERSION')

'11.4.1'

In [3]:
from bonito_pulled.bonito.util import __data__, __models__, __dir__
# list(__dir__.iterdir())
# __data__

In [2]:
from bonito_pulled.bonito.cli.download import main
# main({})

In [1]:
from bonito_pulled.bonito.util import load_model
from bonito_pulled.bonito.util import __models__

#TODO there are multiple models, this one is _fast
dirname = __models__/'dna_r10.4.1_e8.2_fast@v3.5.1'
device = 'cuda'
model = load_model(
    dirname, 
    device, 
    weights=None, 
    half=None, 
    chunksize=None, 
    batchsize=None, 
    overlap=None, 
    quantize=False, 
    use_koi=False
)

print(type(model))
model

<class 'bonito_pulled.bonito.crf.model.Model'>


Model(
  (encoder): Serial(
    (0): Convolution(
      (conv): Conv1d(1, 4, kernel_size=(5,), stride=(1,), padding=(2,))
      (activation): Swish()
    )
    (1): Convolution(
      (conv): Conv1d(4, 16, kernel_size=(5,), stride=(1,), padding=(2,))
      (activation): Swish()
    )
    (2): Convolution(
      (conv): Conv1d(16, 96, kernel_size=(19,), stride=(5,), padding=(9,))
      (activation): Swish()
    )
    (3): Permute()
    (4): LSTM(
      (rnn): LSTM(96, 96)
    )
    (5): LSTM(
      (rnn): LSTM(96, 96)
    )
    (6): LSTM(
      (rnn): LSTM(96, 96)
    )
    (7): LSTM(
      (rnn): LSTM(96, 96)
    )
    (8): LSTM(
      (rnn): LSTM(96, 96)
    )
    (9): LinearCRFEncoder(
      (linear): Linear(in_features=96, out_features=256, bias=True)
      (activation): Tanh()
    )
  )
)

In [30]:
# model = model.encoder[:-1] #SKipping the CRF encoder
import torch
from bonito_pulled.bonito.nn import Permute

seq_model = torch.nn.Sequential(
    model,
    Permute((1,2,0)),
    torch.nn.MaxPool1d(2000), #maxpooling over the whole RNN length, instead do convolution maybe?
    torch.nn.Flatten(),
    torch.nn.Linear(320, 1),
).to('cuda').half()
x = torch.rand(32,1,10000).to('cuda').half()
seq_model(x).shape

torch.Size([32, 1])

In [2]:
import torch
#TODO batch size deosnt matteR? always 1536
x = torch.rand(32,1,10000).to('cuda').half()
out = model(x)
# out = model.decode(x)
out.shape
#Current problem - why is the first dimension changed from 32 (batch size)
# Conv - ok, permute doesnt run? CRF_encoder OK, => rnn or permute
# ? : Reduced sequence length : features
# out

torch.Size([2000, 32, 320])